# Neighborhood Computations

In [191]:
import numpy as np
import igl
import meshplot
import ipywidgets as widgets

In [192]:
bunny_v, bunny_f = igl.read_triangle_mesh("data/bunny.off")
cube_v, cube_f = igl.read_triangle_mesh("data/cube.obj")
sphere_v, sphere_f = igl.read_triangle_mesh("data/sphere.obj")

In [193]:
meshplot.plot(bunny_v, bunny_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [194]:
meshplot.plot(cube_v, cube_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [195]:
meshplot.plot(sphere_v, sphere_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Vertex-to-Face Relations

In [196]:
def vertex_to_face(v, f, mesh):
    faces, vertices = igl.vertex_triangle_adjacency(f, len(v))
    
    v_to_f_list = [faces[vertices[i] : vertices[i+1]] for i in range (len(vertices) - 1)]
    print(mesh)
    for v, f in enumerate(v_to_f_list) :
        print(f"Vertex {v}: Faces {f}")

vertex_to_face(bunny_v, bunny_f, "BUNNY")
vertex_to_face(cube_v, cube_f, "CUBE")
vertex_to_face(sphere_v, sphere_f, "SPHERE")

BUNNY
Vertex 0: Faces [849 850 912 944 945]
Vertex 1: Faces [ 738  788 1278 1833]
Vertex 2: Faces [ 248 2795 2857 2972 3408]
Vertex 3: Faces [911 912 945 960 974 991]
Vertex 4: Faces [ 971 2977 2984 3012 3013]
Vertex 5: Faces [1455 1658 1706 2471 4816 4884]
Vertex 6: Faces [1906 2443 2506 2518 2526]
Vertex 7: Faces [   5   15  103 2576 2697 2799]
Vertex 8: Faces [  55  118  144 2711 3005]
Vertex 9: Faces [2599 2729 2805 2945 3066]
Vertex 10: Faces [  96  941 2736 2773 2803 2832 3164 3165]
Vertex 11: Faces [  13   96 1082 1689 2736 3137 3143]
Vertex 12: Faces [1689 3137 3816 3911]
Vertex 13: Faces [1037 1053 1054 1087 1122 1133 1134]
Vertex 14: Faces [ 90 107 123 139]
Vertex 15: Faces [1017 1036 1053 1074 1121 1122]
Vertex 16: Faces [ 712  713  728  737  750 1970 2034]
Vertex 17: Faces [ 680  713  728 3078]
Vertex 18: Faces [ 407  515 2113 2114 2246 2335 3402]
Vertex 19: Faces [ 99 100 164 209 884 886 984]
Vertex 20: Faces [ 409  410  411 1540 3177 3178 3237]
Vertex 21: Faces [ 410  411

## Vertex-to-Vertex Relations

In [197]:
def vertex_to_vertex(v, f, mesh):
    v_to_v_list = igl.adjacency_list(f)
    
    print(mesh)
    for vertex, vertices in enumerate(v_to_v_list):
        print(f"Vertex {vertex}: Vertices {vertices}")

vertex_to_vertex(bunny_v, bunny_f, "BUNNY")
vertex_to_face(cube_v, cube_f, "CUBE")
vertex_to_vertex(sphere_v, sphere_f, "SPHERE")

BUNNY
Vertex 0: Vertices [3, 24, 308, 525, 542]
Vertex 1: Vertices [415, 549, 551, 596]
Vertex 2: Vertices [134, 287, 465, 497, 1308]
Vertex 3: Vertices [0, 246, 308, 510, 525, 543]
Vertex 4: Vertices [406, 1235, 1368, 1371, 1375]
Vertex 5: Vertices [792, 917, 1113, 3426, 3454, 3482]
Vertex 6: Vertices [484, 1130, 1163, 1179, 1191]
Vertex 7: Vertices [1236, 1267, 1352, 1381, 1382, 2146]
Vertex 8: Vertices [74, 1325, 1365, 1402, 1420]
Vertex 9: Vertices [280, 465, 1225, 1227, 1259]
Vertex 10: Vertices [11, 349, 451, 1319, 1524, 1537, 1565, 1671]
Vertex 11: Vertices [10, 12, 349, 1413, 1524, 1542, 1693]
Vertex 12: Vertices [11, 1542, 1693, 1724]
Vertex 13: Vertices [15, 238, 246, 266, 292, 526, 543]
Vertex 14: Vertices [156, 390, 440, 889]
Vertex 15: Vertices [13, 480, 503, 511, 526, 543]
Vertex 16: Vertices [17, 294, 324, 549, 551, 796, 1217]
Vertex 17: Vertices [16, 103, 324, 1217]
Vertex 18: Vertices [1547, 1598, 2014, 2034, 2053, 2070, 2085]
Vertex 19: Vertices [267, 343, 378, 506, 1

## Shading

Meshplot requires per vertex normals, so we need to "explode" the mesh

### Flat Shading

In [198]:
def explode_mesh (v, f):
    exploded_v = []
    exploded_f = []
    
    vertex = 0
    for face in f: 
        new_f = []

        for vertice in face: 
            v_vector = v[vertice]
            exploded_v.append(v_vector)
            new_f.append(vertex)
            vertex += 1
        exploded_f.append(new_f)

    exploded_v = np.array(exploded_v)
    exploded_f = np.array(exploded_f)

    return exploded_v, exploded_f

In [199]:
def flat_shading (v, f, exploded_v, exploded_f):
    # PER FACE NORMALS: return a list of sublist where each sublist is 
    #                   a vector of each face's normal
    per_face_normals = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
    exploded_n = np.repeat(per_face_normals, 3, axis=0)

    p = meshplot.plot(exploded_v, exploded_f, n=exploded_n, shading={"flat": False})
    # p.add_lines(exploded_v, exploded_v + 0.01*exploded_n)

bunny_exp_v, bunny_exp_f = explode_mesh(bunny_v, bunny_f)
flat_shading(bunny_v, bunny_f, bunny_exp_v, bunny_exp_f)

cube_exp_v, cube_exp_f = explode_mesh(cube_v, cube_f)
flat_shading(cube_v, cube_f, cube_exp_v, cube_exp_f)

sphere_exp_v, sphere_exp_f = explode_mesh(sphere_v, sphere_f)
flat_shading(sphere_v, sphere_f, sphere_exp_v, sphere_exp_f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Per-vertex Shading

In [200]:
bunny_normals = igl.per_vertex_normals(bunny_v, bunny_f)
p = meshplot.plot(bunny_v, bunny_f, n=bunny_normals, shading={"flat": False})

cube_normals = igl.per_vertex_normals(cube_v, cube_f)
p = meshplot.plot(cube_v, cube_f, n=cube_normals, shading={"flat": False})

sphere_normals = igl.per_vertex_normals(sphere_v, sphere_f)
p = meshplot.plot(sphere_v, sphere_f, n=sphere_normals, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Per-corner Shading

In [201]:
def corner_shading (v, f, exploded_v, exploded_f):
    # PER FACE NORMALS: return a list of sublist where each sublist is 
    #                   a vector of each face's normal
    corner_threshold = 30.0
    corner_normals = igl.per_corner_normals(v, f, corner_threshold)

    threshold = np.deg2rad(corner_threshold)

    faces, vertices = igl.vertex_triangle_adjacency(f, len(v))
    
    v_to_f_list = [faces[vertices[i] : vertices[i+1]] for i in range (len(vertices) - 1)]
    
    adjacent_faces = []
    for fc in f:
        f_adjacent = []
        for v in fc:
            v_adjacent = v_to_f_list[v]
            f_adjacent.extend(v_adjacent)

        f_adjacent = list(set(f_adjacent)) # remove duplicate faces
        adjacent_faces.append(f_adjacent)
        
    # SHARP EDGES
    shading_normals = []
    for i, face in enumerate(f):
        face_corner_normals = corner_normals[face]
        new_normal = []
        
        # Angle between two adjacent corner 
        for i in range(3):
            n1 = face_corner_normals[i]
            n2 = face_corner_normals[(i+1)%3]

            angle = np.arccos(np.clip(np.dot(n1, n2), -1.0, 1.0))
            # print(angle)

            if angle < threshold:
                new_n = (n1 + n2) / 2
                # new_n /= np.linalg.norm(new_n) 
                new_normal.append(new_n)
            else:
                new_normal.append(n1)
    
        shading_normals.append(new_normal)

    shading_normals = np.array(shading_normals)
    
    meshplot.plot(exploded_v, exploded_f, n=shading_normals, shading={"flat": False})
    
corner_shading(bunny_v, bunny_f, bunny_exp_v, bunny_exp_f)
corner_shading(cube_v, cube_f, cube_exp_v, cube_exp_f)
corner_shading(sphere_v, sphere_f, sphere_exp_v, sphere_exp_f)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Connected Components

In [202]:
car_v, car_f = igl.read_triangle_mesh("data/car.off")
cup_v, cup_f = igl.read_triangle_mesh("data/coffeecup.off")

def connected_components(v, f, mesh):
    components = igl.face_components(f)
    # return list of component label for each face 
    # same label means the faces belong to the same component

    print(mesh)

    # Number of Connected Components
    num_components = np.max(components) + 1 # plus 1 for the "0" component
    print(f"Number of Connected Components: {num_components}\n")

    # Number of Faces of Each Component
    size_components = {}
    for c in components:
        if c in size_components:
            size_components[c] += 1
        else:
            size_components[c] = 1

    for i in range(num_components):
        print(f"Component {i} has {size_components[i]} faces")

    # Generate Colors for Each Components
    colors = np.random.rand(num_components, 3)
    color_component = colors[components]

    meshplot.plot(v, f, c=color_component)  

connected_components(car_v, car_f, "CAR")
connected_components(cup_v, cup_f, "COFFEE CUP")


CAR
Number of Connected Components: 11

Component 0 has 90 faces
Component 1 has 192 faces
Component 2 has 192 faces
Component 3 has 13216 faces
Component 4 has 704 faces
Component 5 has 1088 faces
Component 6 has 1088 faces
Component 7 has 1088 faces
Component 8 has 1088 faces
Component 9 has 736 faces
Component 10 has 736 faces


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.194198…

COFFEE CUP
Number of Connected Components: 2

Component 0 has 3360 faces
Component 1 has 2304 faces


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-3.253649…

## A simple subdivision scheme

In [203]:
def subdivision(v, f):
    # MIDPOINT OF FACES 
    midpoints = []
    for face in f:
        m = 1/3 * sum(v[i] for i in face) # barycenter
        midpoints.append(m)
    
    midpoints = np.array(midpoints)
    new_v = np.vstack([v, midpoints])
    # new vertices are added at the end of original v array
        
    p1 = meshplot.plot(v, f, shading={"wireframe": True})
    # p1.add_points(midpoints, shading={"point_size": 0.1, "color": "red"})

    # CONNECT MIDPOINTS, ADD NEW FACES
    num_vertices = len(v)
    
    new_f = []
    for face_i, face in enumerate(f):
        midpoint_i = num_vertices + face_i
        # each midpoint is added at the end of v array by face's order
        new_f.append([face[0], face[1], midpoint_i])
        new_f.append([face[0], face[2], midpoint_i])
        new_f.append([face[1], face[2], midpoint_i])

    new_f = np.array(new_f)
    p2 = meshplot.plot(new_v, new_f, shading={"wireframe": True})
    # p2.add_points(midpoints, shading={"point_size": 0.1, "color": "red"})

    # FLIP EDGES
    new_pos = []
    adjacent_v = igl.adjacency_list(f)
    for i, vert in enumerate(adjacent_v):
        n = len(vert)
        a_n = (4 - 2*np.cos(2*np.pi/n))/9
        p = (1 - a_n)* v[i] + (a_n / n) * np.sum([v[j] for j in vert], axis = 0)
        new_pos.append(p)

    new_pos = np.array(new_pos)
    new_pos = np.vstack([new_pos, midpoints])
    
    flipped_f = []
    edge, faces, mapping = igl.edge_topology(v, f)
    # print(len(edge), len(faces), len(c))
    # print(edge_to_faces)
    for i in range(len(edge)):
        e = edge[i]
        fc = mapping[i]
        if (-1) not in fc: # not boundary edge 
            # print(edge)
            e_v1 = e[0]
            e_v2 = e[1] 
            i_m1 = num_vertices + fc[0]
            i_m2 = num_vertices + fc[1]
            flipped_f.append([e_v1, i_m1, i_m2])
            flipped_f.append([e_v2, i_m1, i_m2])

    flipped_f = np.array(flipped_f)

    p3 = meshplot.plot(new_pos, flipped_f, shading={"wireframe": True})
    # p3.add_points(midpoints, shading={"point_size": 0.1, "color": "red"})

subdivision(bunny_v, bunny_f)
subdivision(cube_v, cube_f)
subdivision(sphere_v, sphere_f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016826…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(2.9802322…